In [1]:
from datasets import load_dataset
ds = load_dataset("rakshya34/filtered_voice_english_v1.9")

/home/pdnguyen/miniconda3/envs/ensemble/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import csv
import json
import pandas as pd
import re
import os

#Tách tên .wav + duration từu .csv gốc tạo thành 1 tuple
def extract_file_info(csv_path):
    """
    Input: File .csv có các trường như file,text,duration
    Output: List tuple (file_name, duration)
    """
    file_info_list = []
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            file_path = row['file']
            duration = float(row['duration'])
            file_name = file_path.split('/')[-1]
            file_info_list.append((file_name, duration))

    return file_info_list

def get_duration(file_name, file_info_list):
    """
    Trả về duration của file_name từ file_info_list.
    
    Parameters:
    file_name (str): Tên file cần tìm (vd: '542.wav').
    file_info_list (list): Danh sách các tuple chứa tên file và duration.
    Returns:
    float: Thời lượng của file nếu tìm thấy, None nếu không tìm thấy.
    """
    for file, duration in file_info_list:
        if file == file_name:
            return duration
    return None

def extract_and_save_csv(input_csv_path, output_csv_path):
    """
    Trích xuất các trường cần thiết từ file CSV ban đầu và lưu vào file CSV mới.

    Parameters:
    input_csv_path (str): Đường dẫn tới file CSV ban đầu.
    output_csv_path (str): Đường dẫn tới file CSV mới sẽ lưu.

    Returns:
    None
    """
    with open(input_csv_path, newline='', encoding='utf-8') as infile, open(output_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = ['file', 'text', 'duration']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        writer.writeheader()
        for row in reader:
            name_wav = f"replace/{extract_filename(row['file'])}"
            new_row = {
                'file': name_wav,
                'text': row['text_bartpho'],
                'duration': row['duration']
            }
            writer.writerow(new_row)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]',' ', text)

def clean_text(text):
    """ Chuẩn hóa từ bằng cách chuyển về viết thường và xóa bỏ dấu cách (space) dư thừa

    Args:
        text(str): Câu đầu vào
    Returns:
        (str): Câu sau khi được chuẩn hóa
    """
    text = text.lower()
    text = remove_punctuation(text)
    text = ' '.join(text.split())
    return text 

def extract_filename(file_path):
    """
    Input:
        /mnt/driver/pdnguyen/studen_annoted/data_telesale_extract_10_dir/extra_0/wavs/1620.wav
    Output:
        1620.wav
    """
    return os.path.basename(file_path)

In [17]:
import os
metadata = [
    {
        "audio_filepath": f"/home/pdnguyen/Ensemble_confidence_Nemo/English_data/{path_wav}",
        "text": clean_text(ds["train"]["transcription"][int(path_wav.split(".")[0])]),
        "duration": 10
    }
    for path_wav in os.listdir("/home/pdnguyen/Ensemble_confidence_Nemo/English_data")
]


In [18]:
with open("metadata.json", "w") as file:
    for entry in metadata:
        json.dump(entry, file)
        file.write('\n')